In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.5. GMCSF Responder Prediction/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.5. GMCSF Responder Prediction/Output')
cur_date = "050522"

library(readxl)
library(tidyverse)
library(caret)
library(e1071)
library(Hmisc)
library(glmnet)
library(pROC)
library(randomForest)

#reading in files
molecular_feature_df = read_csv("20220217_mf_qrilc_w40.csv") %>%
     rename(Smoking_Status = sample)
cytokine_df = read_csv("021422_imputed_cytokine_data_w_Ecig40.csv") %>%
     rename(Smoking_Status = `...1`)
demographics_data = data.frame(read_excel("SubjectInfo_022822.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Formula


Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: Matrix


Attaching package: ‘Mat

In [2]:
head(molecular_feature_df)
head(cytokine_df)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NS_070,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
NS_110,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
NS_143,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
NS_152,21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
NS_187,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
NS_196,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


Smoking_Status,IFNg,IL10,IL12p70,IL13,IL1b,IL2,IL4,IL6,TNFa,⋯,Eotaxin,Eotaxin3,IL8,IP10,MCP10,MCP4,MDC,MIP1a,MIP1b,TARC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ecig_39,5.3695611,2.9928536,1.6700162,48.36813,24.117761,5.2657189,0.42455866,13.075670,5.9501019,⋯,358.19662,0.06183719,18547.74,8736.7132,112.48249,10.072081,31.69299,22.02667,46.80070,20.541303
Ecig_41,11.6128116,9.0600267,3.3710810,67.96322,167.431877,12.6837712,0.71329848,19.032378,11.3628057,⋯,267.15489,69.25681747,109246.09,7708.0228,255.29369,13.009597,221.57141,52.67214,149.02215,24.715188
Ecig_44,4.1603189,2.0340947,1.5655687,50.77121,8.870868,2.4837514,0.49649954,9.266373,6.3306852,⋯,103.50569,8.72453305,11221.13,7109.3784,147.73303,1.090400,167.32860,46.98751,101.25635,35.541708
Ecig_45,0.8881171,0.4310717,0.3063518,25.23906,40.477650,0.5200777,0.02914528,6.395885,1.9479068,⋯,298.85050,8.43896238,14633.80,3045.9940,95.82647,21.200301,196.12405,32.89026,24.92311,19.708475
Ecig_47,1.4481966,0.4997130,0.8622316,36.81471,15.755858,0.7766565,0.08862797,5.244501,3.5847722,⋯,67.18448,5.48717948,12078.16,997.9376,66.13370,7.189953,139.46181,16.70862,21.35656,9.885379
Ecig_48,0.8494440,0.4240552,0.2785117,14.51272,9.211680,0.2684367,0.14116411,2.476904,0.5468069,⋯,25.07486,2.15157171,4880.69,246.9985,57.50348,3.359783,69.35689,9.56433,15.67118,9.302557


In [3]:
# trimming down demographics df
demographics_data = demographics_data %>%
    select(c("SubjectID", "Sex")) %>%
    rename(Smoking_Status = SubjectID) %>%
    #necessary to put it into factor and relevel, so algorithm understands which data is considered baseline
    mutate(Sex = relevel(factor(ifelse(Sex == "M", 1, 0)), ref = "0"))

head(demographics_data)

,Smoking_Status,Sex
,<chr>,<fct>
1,Ecig_214,0
2,Ecig_221,1
3,Ecig_222,0
4,Ecig_223,0
5,Ecig_229,0
6,Ecig_230,0


In [4]:
# creating 2 dataframes: with and w/o sex as a covariate
cytokine_df = cytokine_df %>%
    select(c("Smoking_Status", "GMCSF"))

mf_cytokine_df = full_join(molecular_feature_df, cytokine_df) %>%
    select(-Smoking_Status)

mf_cytokine_df_covariates = full_join(molecular_feature_df, cytokine_df) %>%
    full_join(demographics_data) %>%
    select(-Smoking_Status)

head(mf_cytokine_df)
head(mf_cytokine_df_covariates)

Joining, by = "Smoking_Status"
Joining, by = "Smoking_Status"
Joining, by = "Smoking_Status"


MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,MF_14,⋯,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976,GMCSF
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,20.80616,⋯,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593,0.74883419
21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,19.71124,⋯,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881,0.52203974
13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,18.19551,⋯,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912,0.71837615
21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,17.30567,⋯,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788,0.04230443
19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,21.33752,⋯,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550,0.04038862
22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,19.51343,⋯,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000,0.13681913


MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,MF_14,⋯,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976,GMCSF,Sex
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,20.80616,⋯,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593,0.74883419,0
21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,19.71124,⋯,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881,0.52203974,0
13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,18.19551,⋯,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912,0.71837615,0
21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,17.30567,⋯,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788,0.04230443,0
19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,21.33752,⋯,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550,0.04038862,1
22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,19.51343,⋯,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000,0.13681913,0


In [5]:
rf_regression = function(dataset, outcome, pred_outcome){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV

    errors_rf = data.frame()
    variable_importance_df = data.frame()
    
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        ntree_values = c(50, 250, 500) # number of trees (not sure if we wanted something different)
        p = dim(dataset)[2] - 1 # number of variables in dataset
        mtry_values = c(p/2, sqrt(p), p)

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_MSE = list()
        rf_MSE_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_MSE[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_MSE" = reg_rf_pred_tune[[k]]$mse[ntree_values[j]])
                rf_MSE_df = rbind(rf_MSE_df, rf_MSE[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_mse <- which(rf_MSE_df$OOB_MSE == min(rf_MSE_df$OOB_MSE))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = data_train,
        #need to go back and figure out how the number of trees/ variables make a difference even though
        #the errors are the same!!!!
                               ntree = rf_MSE_df$Tree.Number[min(best_mse)], 
                               mtry = rf_MSE_df$Variable.Number[min(best_mse)])

        #predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test)

        # calculating MSE and adding to df
        error_values = postResample(data_test[[pred_outcome]], data_test[[outcome]])
        errors_rf = rbind(errors_rf, error_values[[1]]^2)

        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "MF")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
        
    }
    
    colnames(errors_rf) = c("MSE")
    errors_rf = errors_rf %>%
        summarise(`CV Error` = mean(MSE), `Std Error` = sd(MSE))
    
    variable_importance_df = variable_importance_df %>%
        group_by(MF) %>%
        summarise(IncNodePurity = mean(IncNodePurity)) %>%
        # sorting by most important variables
        arrange(-IncNodePurity)
  
    # return training set, matrix and variable importance values
    return(list(data_train, errors_rf, variable_importance_df))

}

# calling fn
rf_values = rf_regression(mf_cytokine_df, "GMCSF", "pred_GMCSF")
rf_values_covariates = rf_regression(mf_cytokine_df_covariates, "GMCSF", "pred_GMCSF")

In [6]:
# viewing MSE results
rf_model_errors = data.frame(Model = c(rep(c("RF Regression"), times = 2)), Covariates = c("No", "Yes"), 
                             rbind(rf_values[[2]], rf_values_covariates[[2]]))
rf_model_errors

# viewing most significant features
head(rf_values[[3]])
head(rf_values_covariates[[3]])

Model,Covariates,CV.Error,Std.Error
<chr>,<chr>,<dbl>,<dbl>
RF Regression,No,14.34616,21.67795
RF Regression,Yes,13.66475,19.78345


MF,IncNodePurity
<chr>,<dbl>
MF_22782,25.68346
MF_17636,17.60964
MF_5705,13.45294
MF_1404,11.98006
MF_7198,11.09134
MF_3176,10.73420


MF,IncNodePurity
<chr>,<dbl>
MF_22782,15.56781
MF_17636,13.75168
MF_10961,13.38901
MF_3176,12.08999
MF_874,11.63623
MF_7198,11.62545


Rerunning RF model with noise variables to determine, which predictors are significant above the background noise.

In [7]:
train_vars_noise = rf_values[[1]]
train_vars_noise_covariates = rf_values_covariates[[1]]

noise_df = function(train_vars_noise){
    set.seed(7)
    # Add random noise predictors as an additional method to evaluate model performance
    # Adding a column that contains randomly shuffled values from one of the molecules; sampling with replacement
    train_vars_noise$noise1 = sample(train_vars_noise[[colnames(train_vars_noise[3])]], replace = TRUE) 
    train_vars_noise$noise2 = sample(train_vars_noise[[colnames(train_vars_noise[4])]], replace = TRUE)
    train_vars_noise$noise3 = sample(train_vars_noise[[colnames(train_vars_noise[5])]], replace = TRUE)
    train_vars_noise$noise4 = sample(train_vars_noise[[colnames(train_vars_noise[6])]], replace = TRUE)
    train_vars_noise$noise5 = sample(train_vars_noise[[colnames(train_vars_noise[7])]], replace = TRUE)
    
    return(train_vars_noise)
}

# calling fn
noise_training_rf_dataset = noise_df(train_vars_noise)
noise_training_rf_dataset_covariates = noise_df(train_vars_noise_covariates)
head(noise_training_rf_dataset)

MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,MF_14,⋯,MF_68286,MF_73340,MF_77165,MF_78976,GMCSF,noise1,noise2,noise3,noise4,noise5
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,20.80616,⋯,19.13556,16.93182,17.55702,17.05593,0.74883419,22.55244,21.20241,16.64876,21.24913,22.49399
19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,21.33752,⋯,19.41757,16.88993,19.35678,18.95550,0.04038862,21.63219,19.74681,18.64912,22.30371,22.18006
21.50216,23.34553,21.53633,17.93372,24.26751,21.71412,21.32122,21.66214,21.71152,17.86307,⋯,18.52501,17.61326,17.74147,16.76017,0.11269318,21.68604,17.85119,15.36472,21.90605,17.94234
21.97526,24.05067,22.28614,17.23683,18.87955,22.30371,16.31152,22.24701,22.18391,20.20701,⋯,14.56269,16.50120,18.56707,18.43693,0.03916000,23.21817,18.23263,17.50820,21.79253,22.68896
20.01045,16.64943,19.91826,17.22811,17.78279,18.82692,17.24179,20.00943,20.08843,16.34883,⋯,16.05914,19.38379,16.34264,17.44038,0.44144111,22.75360,16.08028,17.88701,21.64130,19.33463
22.53688,24.43968,23.28045,19.36843,18.44241,22.80038,22.39723,17.57684,22.85906,20.30357,⋯,17.13399,17.69150,16.35047,17.86477,0.12937836,19.68977,17.54753,17.50820,21.45922,21.00517


In [8]:
# calling rf fn
noise_rf_values = rf_regression(noise_training_rf_dataset, "GMCSF", "pred_GMCSF")
noise_rf_values_covariates = rf_regression(noise_training_rf_dataset_covariates, "GMCSF", "pred_GMCSF")

In [9]:
# viewing MSE results
noise_rf_model_errors = data.frame(Model = c(rep(c("RF Regression"), times = 2)), Covariates = c("No", "Yes"),  
                             rbind(noise_rf_values[[2]], noise_rf_values_covariates[[2]]))
noise_rf_model_errors

# viewing most significant features
head(noise_rf_values[[3]])
head(noise_rf_values_covariates[[3]])

Model,Covariates,CV.Error,Std.Error
<chr>,<chr>,<dbl>,<dbl>
RF Regression,No,16.71028,21.63989
RF Regression,Yes,16.21376,22.09005


MF,IncNodePurity
<chr>,<dbl>
MF_22782,12.15155
MF_874,12.04676
MF_3176,10.71323
MF_18012,10.59023
MF_7198,10.49688
MF_7476,10.32712


MF,IncNodePurity
<chr>,<dbl>
MF_22782,15.804172
MF_874,14.130434
MF_1496,8.576510
MF_35145,8.440380
MF_12220,8.361231
MF_3324,7.345242


Calculating the average noise rank to determine, which predictors ranked higher than the highest noise rank.

In [10]:
#calculating avg noise rank out of all variables
noise_importance_values = noise_rf_values[[3]] %>%
    column_to_rownames(var = "MF")
noise_importance_values_covariates = noise_rf_values_covariates[[3]] %>%
    column_to_rownames(var = "MF")

In [11]:
highest_noise_rank_fn = function(noise_importance_values){
    #this gives us values between 0 and 1
    #the closer the number gets to 1 the lower it ranks, so the number closest to 1 is the max rank
    noise1rank = which(rownames(noise_importance_values) == "noise1")/nrow(noise_importance_values)
    noise2rank = which(rownames(noise_importance_values) == "noise2")/nrow(noise_importance_values)
    noise3rank = which(rownames(noise_importance_values) == "noise3")/nrow(noise_importance_values)
    noise4rank = which(rownames(noise_importance_values) == "noise4")/nrow(noise_importance_values)
    noise5rank = which(rownames(noise_importance_values) == "noise5")/nrow(noise_importance_values)

    #figuring out which is the highest mean decrease gini or background noise importance value
    highest_noise_rank = min(noise1rank, noise2rank, noise3rank, noise4rank, noise5rank)
    
    return(highest_noise_rank)
}

#calling fn
highest_noise_rank = highest_noise_rank_fn(noise_importance_values)
highest_noise_rank_covariates = highest_noise_rank_fn(noise_importance_values_covariates)
highest_noise_rank
highest_noise_rank_covariates

[1] 0.1416043

[1] 0.3929861

In [12]:
last_significant_rank_fn = function(noise_importance_values, highest_noise_rank){
    for (i in 1:length(noise_importance_values$IncNodePurity)){
    #if the row number (index)/# of variables is less than the highest noise rank the loop will continue
        if (i/length(noise_importance_values$IncNodePurity) > highest_noise_rank){
            last_significant_var = i - 1
            break
        }
    }    
    return(last_significant_var)
}

#calling fn
last_significant_rank = last_significant_rank_fn(noise_importance_values, highest_noise_rank)
last_significant_rank_covariates = last_significant_rank_fn(noise_importance_values_covariates, highest_noise_rank_covariates)
last_significant_rank
last_significant_rank_covariates

[1] 549

[1] 1524

The dataset containing all the features contains 804 MFs above background, while the dataset without nicotine and cotinine has 84. This means that the exclusion of these 2 features forces the model to find erroneous connections between the noise variables and the outcome and that these 2 features are driving much of the model's predictive ability.

In [13]:
# now getting those significant MFs
significant_MFs = rownames(noise_importance_values)[1:last_significant_rank]
significant_MFs_covariates = rownames(noise_importance_values_covariates)[1:last_significant_rank_covariates]

In [14]:
# converting row names back to columns to make next step easier
noise_importance_values = noise_importance_values %>%
    rownames_to_column(var = "MF")
noise_importance_values_covariates = noise_importance_values_covariates %>%
    rownames_to_column(var = "MF")

In [15]:
# getting dataframe ready to export variable importance ranks from above and whether those features passed the 
# background filter
noise_variable_importance_ranks = function(noise_importance_values_df, significant_predictors){
    # filtering the original rank df for predictors that were significant (above random noise)
    filtered_significant_ranks_df = noise_importance_values_df %>%
        filter(MF %in% significant_predictors) %>%
        arrange(-IncNodePurity)
    
    # adding a column denoting if the feature was above random noise
    filtered_significant_ranks_df$Passed_Filter = c(rep(c("Yes"), times = length(filtered_significant_ranks_df$MF)))
    
    final_df = full_join(filtered_significant_ranks_df, noise_importance_values_df)
    return(final_df)
}

# calling fn
significant_predictors_df = noise_variable_importance_ranks(noise_importance_values, significant_MFs)
significant_predictors_df_covariates = noise_variable_importance_ranks(noise_importance_values_covariates, significant_MFs_covariates)
head(significant_predictors_df)

Joining, by = c("MF", "IncNodePurity")
Joining, by = c("MF", "IncNodePurity")


,MF,IncNodePurity,Passed_Filter
,<chr>,<dbl>,<chr>
1,MF_22782,12.15155,Yes
2,MF_874,12.04676,Yes
3,MF_3176,10.71323,Yes
4,MF_18012,10.59023,Yes
5,MF_7198,10.49688,Yes
6,MF_7476,10.32712,Yes


In [16]:
# exporting results
# feature ranking w/ noise variables
write.csv(significant_predictors_df, paste0(Output,"/", cur_date, "_Feature_Ranking_noise.csv"), row.names = FALSE)
write.csv(significant_predictors_df_covariates, paste0(Output,"/", cur_date, "_Feature_Ranking_covariates_noise.csv"), row.names = FALSE)

#exporting MSE values
write.csv(rf_model_errors, paste0(Output,"/", cur_date, "_GMCSF_prediction_errors.csv"), row.names = FALSE)